# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
#import packages
import pandas as pd
from scipy.stats import ttest_ind
import numpy as np
from scipy.stats import t as tdist
#load data
df = pd.read_csv('homepage_actions.csv')
#Explore the 'id' column
print(df['id'].describe())

count      8188.000000
mean     564699.749878
std      219085.845672
min      182988.000000
25%      373637.500000
50%      566840.500000
75%      758078.000000
max      937217.000000
Name: id, dtype: float64


In [5]:
# Check how many viewers also clicked
viewers = df[df['action'] == 'view']['id'].unique()
clickers = df[df['action'] == 'click']['id'].unique()
# Number of viewers who also clicked
viewers_clicked = len(set(viewers).intersection(clickers))
viewers_clicked

1860

In [6]:
# Check for anomalies: Did anyone click who didn't view?
clickers_without_view = len(set(clickers) - set(viewers))
clickers_without_view

0

In [11]:
# Check for overlap between control and experiment groups
control_group = df[df['group'] == 'control']['id'].unique()
experiment_group = df[df['group'] == 'experiment']['id'].unique()
# Number of users in both control and experiment groups
overlap_users = len(set(control_group).intersection(experiment_group))
overlap_users

0

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [12]:
# Calculate the click-through rate (CTR) for each group
ctr_control = df[(df['group'] == 'control') & (df['action'] == 'click')]['id'].nunique() / len(control_group)
ctr_experiment = df[(df['group'] == 'experiment') & (df['action'] == 'click')]['id'].nunique() / len(experiment_group)

In [13]:
# Perform a t-test
t_stat, p_value = ttest_ind(
    df[(df['group'] == 'control') & (df['action'] == 'click')]['id'].unique(),
    df[(df['group'] == 'experiment') & (df['action'] == 'click')]['id'].unique(),
    equal_var=False  # Welch's t-test for unequal variances
)
# Display results
print(f"Click-Through Rate (CTR) - Control Group: {ctr_control}")
print(f"Click-Through Rate (CTR) - Experiment Group: {ctr_experiment}")
print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")

# Check for statistical significance
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis. The experimental homepage is more effective than the control group.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference between the groups.")

Click-Through Rate (CTR) - Control Group: 0.2797118847539016
Click-Through Rate (CTR) - Experiment Group: 0.3097463284379172
T-statistic: -0.5827851039715325
P-value: 0.5601087216132623
Fail to reject the null hypothesis. There is no significant difference between the groups.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [14]:
# Calculate the overall click-through rate (CTR)
overall_ctr = df[df['action'] == 'click']['id'].nunique() / df['id'].nunique()
# Calculate the expected number of clicks for the experiment group
expected_clicks_experiment = len(experiment_group) * overall_ctr
print(f"Overall Click-Through Rate (CTR): {overall_ctr}")
print(f"Expected Clicks in Experiment Group: {expected_clicks_experiment}")

Overall Click-Through Rate (CTR): 0.2939317319848293
Expected Clicks in Experiment Group: 880.6194690265486


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [16]:
import math
experiment_clicks = df[(df['group'] == 'experiment') & (df['action'] == 'click')]['id'].nunique()

# Calculate the standard deviation of a binomial distribution
std_dev_binomial = math.sqrt(len(experiment_group) * overall_ctr * (1 - overall_ctr))

# Calculate the number of standard deviations
num_std_dev = (experiment_clicks - expected_clicks_experiment) / std_dev_binomial
num_std_dev

1.9001261506527332

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [18]:
from scipy.stats import norm

# Calculate the p-value using the cumulative distribution function (CDF)
p_value_normal = norm.cdf(num_std_dev)

p_value_normal

0.9712917166699202

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.